# Tensorとは？
---

前単元の復習にもなるが、再度Tensor型の特徴を紹介する。

1. TensorはNumPyのndarrayに似ている
2. GPUを使って演算を行うことが可能
3. 自動微分という機能がある

前単元では、特徴1にあたる、`Tensor`と`numpy.ndarray`の共通点などを学習した。  
本単元では`numpy.ndarray`ではできない、`Tensor`の特徴**2**と**3**を順に学んでいこう！


## この単元の目標

* TensorのGPU演算について知る
* Tensorの自動微分機能を学ぶ

  → **Tensor型の特徴を勉強していこう**  


In [ ]:
# pytorchライブラリのインポート
import torch

#### CPUとGPUについて
---

ハードウェアにはそれぞれ得意な計算の種類がある。  
それぞれの得意分野は以下の通りだ。

* if文のような条件分岐が多い複雑な計算は**CPU**
* for文のような繰り返しが多い単純な計算は**GPU**

DL(ディープラーニング)の計算では行列の積和を多く扱う。  
この計算はfor文が多い単純な計算に当たるため、DLの演算は**GPU**と相性がよい。

## GPU対応
---

特徴2：Tensorは**GPUを使って演算を行うことが可能**
* DLの計算はGPUを使うことで圧倒的なパフォーマンスを発揮することができる
    * PyTorchはCPU/GPU切替可能
    * NumpyはGPU非対応


実際にプログラムを実行してTensor型のデータを作成しイメージを掴もう！

---
**※注意※** 本章の例題はGPU環境を整えてから出ないと動かない。  
実行すると「GPUが利用できる環境が整っていません」とでる場合がほとんどだろう。  
本講座ではGPU環境の構築については扱わないため、構築されていればこのように動くのだろうなと感じつつ説明文を読み学んでほしい。

【例題】下記のプログラムを実行して、CPUからGPUデバイスへの切り替えをしてみよう。

In [ ]:
if torch.cuda.is_available():
    gpu = torch.device("cuda")  # GPUデバイスオブジェクトの作成
    cpu = torch.device("cpu")  # CPUデバイスオブジェクトの作成
    data1 = torch.zeros((2, 2), device=gpu)  # GPU上に作成
    data_cpu = data1.to(cpu)  # CPUへ転送
    data_gpu = data_cpu.to(gpu)  # GPUへ転送
    print(data_cpu)
    print(data_gpu)
else:
    print("GPUが利用できる環境が整っていません")

- ```
tensor([[0., 0.],
        [0., 0.]])
tensor([[0., 0.],
        [0., 0.]], device='cuda:0')
```
環境が整っている場合、上記のように表示される。


1行目のif文はGPUが利用できる環境が構築されているか判断する関数だ

上記のような手順で、データをいつでも任意のデバイスへ転送することができる。  
また、GPU上にデータが存在するTensorを出力した場合、`device='cuda:0'`と出力されることがわかる。

* GPUのデバイスオブジェクトの作成：`torch.device("cuda")`
* CPUのデバイスオブジェクトの作成：`torch.device("cpu")`

* デバイス間の転送方法：`Tensor名.to(転送先デバイスオブジェクト)`



* テンソルをGPU上に作成したい場合はオプション`device`を指定する
    * `device=GPUのデバイスオブジェクト`    

---
**※注意※** 本章の例題はGPU環境を整えてから出ないと動かないため、例題で「GPUが利用できる環境が整っていません」と出力された場合は本章の問題は飛ばして次の章へ進もう。

【問題】変数`data2`にtorch.onesで3行2列のデータを作成した後、GPUへデータを転送し出力しよう。

In [ ]:
gpu = torch.device("cuda")
data2 = torch.ones((3, 2))
print(data2.to(gpu))

* 
```
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]], device='cuda:0')
```
と表示できれば成功だ。

## Tensor同士の計算
---
特徴3：Tensorには**自動微分という機能がある**
* Tensor同士の計算は全て記録している
* この機能により、DLプログラムが非常に簡単になる

自動微分機能を体験してみよう！
まずは、Tensor同士の計算をしてみよう！

【例題】Tensor同士の計算を行う。プログラムを実行して実行結果から挙動を確かめよう。

In [ ]:
a = torch.ones(3)
b = torch.rand(3)
print(a)
print(b)
print(a+3)
print(a*3)
print(a+b)

Tensor同士の計算は以上のように行うことができる。

**注意点**
* 形の異なるTensor同士は計算できない
* CPU上のデータとGPU上のデータは計算することができない
    
【問題】変数`c`に`ones()`で3行2列のデータ、変数`d`に`zeros()`で3行2列のデータを作成した後、`c*3+d*2`を計算し出力しよう。

* 
```
tensor([[3., 3.],
        [3., 3.],
        [3., 3.]])
```
と表示できれば成功だ。

## 自動微分
---
それでは、自動微分機能を体験してみよう！

【例題1】自動微分機能を体験しよう。

In [ ]:
x = torch.tensor(1.0, requires_grad = True)
a, b = 3, 5
y = a*x + b

print(y)

* 自動微分機能を利用したい場合は**Tensor**の宣言時に、オプション`requires_grad`を指定する
    * `requires_grad=True`  
        * `True`にすることで計算の追跡(記録)をするよう設定できる
    * 今回は要素をスカラーで値を1としたが、行列でも良いし中身の値も何でも良い。(微分には関係ない)



上記のプログラムでは
$$ y = 3x + 5 $$
の`x=1`のときの出力を表してるので`y=8`

* `grad_fn=<AddBackward0>`は`y`が足し算により算出されたということを示している。

【例題2】自動微分機能を体験するためTensorの演算を行う。

In [ ]:
y.backward()

何も出力されないが、`計算後のTensor名.backward()`で微分が行われた。  
以下のプログラムを動かして確認してみよう。

In [ ]:
print(x.grad)

* `Tensor名.grad`とすることでその変数名の勾配（微分値）がわかる。
* yの式をxで微分すると勾配は3になる。
$$ \frac{dy}{dx} = 3 $$

* **Tensor**で宣言されていない変数は微分できないので注意
    * 上記のプログラムのままでは`a.grad`や`b.grad`は不可能
* この特徴を利用することで、DLのプログラムを楽に構成できるぞ！

【注意】  
自動微分を利用する上で気をつけてほしいポイントがある。以下のプログラムを何度か実行してみてほしい。



In [ ]:
y = a*x + b
y.backward()
print(x.grad)

勾配が累積されていることがわかるだろうか？  
`backward()`が実行されるたびに`Tensor名.grad`は加算を繰り返してしまう。

そのため、ループ文を利用して何度も利用する際には、  
`backward()`の前に**勾配の初期化**という作業が必要になるぞ！(詳しくは「モデルの実装と学習」にて)

【問題】【例題1,2】を参考に以下の式をvとwでそれぞれ偏微分してみよう。

$$ out = 4v + 6w + 1 $$

* 
微分値がそれぞれ**4**と**6**と表示できれば成功だ。

説明を簡単にするため、スカラー値の微分を行ったが多次元行列の微分ももちろんできるぞ！

以上の「GPU対応」「自動微分」を利用できるのがPyTorchのTensor型の大きな特徴だ！